## Criando a tabela relations

Este script tem como objetivo encontrar uma relação de dados entre as tabelas league, country, team_attributes e player e criar uma nova tabela chamada `relations`

#### Import das libs

In [1]:
#Importando das libs que serão utilizadas no processo
import sqlite3
import pandas as pd

#### Conectando ao banco e iniciando o cursor

In [2]:
#conectando ao banco de dados test_analytics_engineer
conn = sqlite3.connect('test_analytics_engineer.db')

In [3]:
#instanciando o cursor
c = conn.cursor()

#### Criando a tabela relations
Para a criação desta tabela apliquei as seguintes transformações:
- Criei uma CTE com visão unificada da tabela match, consolidando todos os jogadores e times em uma única respectiva coluna. Ex: os times estavam separados por home e away e foram consolidados em uma única coluna chamada team_api_id
- A CTE mencionada acima é a que servirá de ligação entre as tabelas refined_player, refined_country e refined_league
- O objetivo inicial era avaliar o desempenho do jogador de acordo com seu peso e altura e os atributos do time que estão presentes na refined_team_attributes. Porém, conforme identificado no script anterior de exploração dos dados, esta tabela não aparenta está com informações consistentes. Por este motivo, a avaliação de performance será realizada através da tabela de matches, identificando número de vitórias do time

In [5]:
c.execute('''
    CREATE TABLE IF NOT EXISTS relations AS
    WITH home_matches AS (
        SELECT match_api_id, country_id, league_id, home_team_api_id AS team_api_id, home_player_1 AS player_api_id, home_team_wins AS win, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, home_team_api_id, home_player_2, home_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, home_team_api_id, home_player_3, home_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, home_team_api_id, home_player_4, home_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, home_team_api_id, home_player_5, home_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, home_team_api_id, home_player_6, home_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, home_team_api_id, home_player_7, home_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, home_team_api_id, home_player_8, home_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, home_team_api_id, home_player_9, home_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, home_team_api_id, home_player_10, home_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, home_team_api_id, home_player_11, home_team_wins, draw_match FROM refined_match
    ),
    
        away_matches AS (
        SELECT match_api_id, country_id, league_id, away_team_api_id AS team_api_id, away_player_1 AS player_api_id, away_team_wins AS win, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, away_team_api_id, away_player_2, away_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, away_team_api_id, away_player_3, away_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, away_team_api_id, away_player_4, away_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, away_team_api_id, away_player_5, away_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, away_team_api_id, away_player_6, away_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, away_team_api_id, away_player_7, away_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, away_team_api_id, away_player_8, away_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, away_team_api_id, away_player_9, away_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, away_team_api_id, away_player_10, away_team_wins, draw_match FROM refined_match
        UNION ALL
        SELECT match_api_id, country_id, league_id, away_team_api_id, away_player_11, away_team_wins, draw_match FROM refined_match
    ),
    
    union_matches AS (
        SELECT * FROM home_matches
        UNION ALL
        SELECT * FROM away_matches
    )
    
    SELECT
        union_matches.match_api_id,
        union_matches.country_id,
        refined_country.name_en_us AS country_name,
        union_matches.league_id,
        refined_league.name AS league_name,
        union_matches.team_api_id,
        refined_team.team_long_name AS team_name,
        union_matches.win,
        union_matches.draw_match AS draw,
        CASE 
            WHEN NOT(union_matches.win OR union_matches.draw_match) THEN TRUE ELSE FALSE END AS lose,
        union_matches.player_api_id,
        refined_player.player_name,
        refined_player.height_in_cm,
        refined_player.weight_in_kg
        
    FROM 
        union_matches
    INNER JOIN
        refined_country
    ON union_matches.country_id = refined_country.id
    INNER JOIN
        refined_league
    ON union_matches.league_id = refined_league.id
    INNER JOIN
        refined_team
    ON union_matches.team_api_id = refined_team.team_api_id
    INNER JOIN
        refined_player
    ON union_matches.player_api_id = refined_player.player_api_id
        
''')

In [6]:
#chegando alguns dados
df = pd.read_sql('''   
    SELECT *
    FROM 
        relations
   LIMIT 100
''',conn)
df.head(10)

,match_api_id,country_id,country_name,league_id,league_name,team_api_id,team_name,win,draw,lose,player_api_id,player_name,height_in_cm,weight_in_kg
0,493016,1,Belgium,1,Belgium Jupiler League,9996,Royal Excel Mouscron,0,1,0,39890,Mark Volders,187.96,83.01
1,493017,1,Belgium,1,Belgium Jupiler League,8203,KV Mechelen,1,0,0,38327,Wouter Biebauw,187.96,82.10
2,493018,1,Belgium,1,Belgium Jupiler League,9986,Sporting Charleroi,1,0,0,95597,Bertrand Laquait,185.42,79.83
3,493020,1,Belgium,1,Belgium Jupiler League,9994,Sporting Lokeren,0,0,1,30934,Boubacar Barry Copa,180.34,68.95
4,493021,1,Belgium,1,Belgium Jupiler League,8342,Club Brugge KV,1,0,0,37990,Stijn Stijnen,187.96,83.01
5,493022,1,Belgium,1,Belgium Jupiler League,9993,Beerschot AC,1,0,0,38391,Silvio Proto,185.42,77.11
6,493023,1,Belgium,1,Belgium Jupiler League,7947,FCV Dender EH,1,0,0,39153,Cedric Berthelin,193.04,97.07
7,493024,1,Belgium,1,Belgium Jupiler League,10000,SV Zulte-Waregem,0,1,0,37900,Sammy Bossuyt,185.42,78.02
8,493025,1,Belgium,1,Belgium Jupiler League,9984,KSV Cercle Brugge,0,0,1,36835,Bram Verbist,182.88,88.90
9,493026,1,Belgium,1,Belgium Jupiler League,9998,RAEC Mons,0,0,1,38252,Frederic Herpoel,182.88,83.01


In [7]:
#commitando criação da tabela
conn.commit()

### Realizando algumas análises na tabela relations

#### Quais os jogadores que mais estiveram em partidas que seu time venceu ?

In [16]:
#chegando alguns dados
df = pd.read_sql('''
    WITH players_wins AS (
        SELECT
            team_name,
            player_name,
            SUM(win) AS wins,
            ROW_NUMBER() OVER (PARTITION BY team_name ORDER BY SUM(win) DESC) AS rn
        FROM 
            relations
        GROUP BY
            team_name,
            player_name
    )
    
    SELECT 
        team_name,
        player_name,
        wins
    FROM 
        players_wins
    WHERE
        rn = 1
    ORDER BY 
        wins DESC
''',conn)
df.head(10)

,team_name,player_name,wins
0,FC Barcelona,Lionel Messi,194
1,Real Madrid CF,Cristiano Ronaldo,175
2,FC Bayern Munich,Philipp Lahm,156
3,Celtic,Scott Brown,155
4,Juventus,Gianluigi Buffon,151
5,Chelsea,John Terry,146
6,Manchester United,Wayne Rooney,142
7,Manchester City,Joe Hart,141
8,Olympique de Marseille,Steve Mandanda,141
9,Napoli,Marek Hamsik,135


Estes são os jogadores que mais atuaram por seus clubes em jogos com vitória

#### Quantos jogadores diferentes foram escalados em cada país durante todo o período ?

In [19]:
#chegando alguns dados
df = pd.read_sql('''
    SELECT
        country_name,
        COUNT(DISTINCT player_api_id) AS players
    FROM 
        relations
    GROUP BY
        country_name
    ORDER BY
        players DESC
''',conn)
df.head(10)

,country_name,players
0,Spain,1500
1,France,1468
2,Italy,1421
3,England,1397
4,Portugal,1347
5,Germany,1244
6,Netherlands,1222
7,Belgium,1148
8,Scotland,1018
9,Switzerland,751


A Espanha se destaca como o país que mais teve jogadores diferentes atuando pela liga. Um possível hipótese que justifique este número é o fato de os 2 melhores times do mundo serem do país e com isso o volume de transferências (chegada e saída de jogadores) pode ser mais elevado.

#### Qual é a altura e peso médio dos jogadores dos times que mais vencem ?

In [62]:
#chegando alguns dados
df = pd.read_sql('''
    WITH dedup_matches AS (
        SELECT
            match_api_id,
            team_api_id,
            win
        FROM 
            relations
        GROUP BY
            match_api_id,
            team_api_id,
            win
    ),
    
    team_imc AS (
        SELECT
            team_api_id,
            team_name,
            AVG(height_in_cm) AS height_in_cm_mean,
            AVG(weight_in_kg) AS weight_in_kg_mean,
            AVG(weight_in_kg / ((height_in_cm*0.01*height_in_cm*0.01))) AS imc_mean
        FROM 
            relations
        GROUP BY
            team_api_id,
            team_name
    )
    
    SELECT
        team_imc.team_api_id,
        team_imc.team_name,
        team_imc.height_in_cm_mean,
        team_imc.weight_in_kg_mean,
        team_imc.imc_mean,
        COUNT(DISTINCT dedup_matches.match_api_id) AS matches,
        SUM(dedup_matches.win) AS wins,
        ROUND((SUM(dedup_matches.win) * 1.0 / COUNT(DISTINCT dedup_matches.match_api_id) * 1.0) * 100) AS win_ratio
        
    FROM
        dedup_matches
    INNER JOIN
        team_imc
    ON dedup_matches.team_api_id = team_imc.team_api_id
    GROUP BY
        team_imc.team_api_id,
        team_imc.team_name,
        height_in_cm_mean,
        team_imc.weight_in_kg_mean,
        team_imc.imc_mean
    ORDER BY
        win_ratio DESC
''',conn)
df.head(10)

,team_api_id,team_name,height_in_cm_mean,weight_in_kg_mean,imc_mean,matches,wins,win_ratio
0,8634,FC Barcelona,177.896282,73.784356,23.293333,304,234,77.0
1,9772,SL Benfica,182.171759,77.697806,23.363017,235,179,76.0
2,8633,Real Madrid CF,182.437269,77.698700,23.340458,304,228,75.0
3,9773,FC Porto,182.268990,77.073423,23.188067,236,175,74.0
4,9925,Celtic,182.764338,76.464227,22.867847,304,218,72.0
5,8548,Rangers,183.145849,79.153140,23.585624,152,108,71.0
6,9823,FC Bayern Munich,182.851985,79.262356,23.644950,272,193,71.0
7,8593,Ajax,181.188200,76.332767,23.209599,251,168,67.0
8,8640,PSV,182.781701,78.094608,23.371428,249,165,66.0
9,8635,RSC Anderlecht,183.790635,77.940342,23.028625,189,119,63.0


O Barcelona é um ponto fora da curva neste top 10 de times que mais venceram: o time possui uma média de ~5 cm à menos de altura e ~4 kg à menos de peso. No mais, os times tem uma média entre 182 cm de altura e 77 kg.

In [61]:
#Avaliando peso e altura por faixa
df = pd.read_sql('''
    WITH dedup_matches AS (
        SELECT
            match_api_id,
            team_api_id,
            win
        FROM 
            relations
        GROUP BY
            match_api_id,
            team_api_id,
            win
    ),
    
    team_imc AS (
        SELECT
            team_api_id,
            team_name,
            AVG(height_in_cm) AS height_in_cm_mean,
            AVG(weight_in_kg) AS weight_in_kg_mean
        FROM 
            relations
        GROUP BY
            team_api_id,
            team_name
    )
    
    SELECT
        CASE
            WHEN team_imc.height_in_cm_mean <= 170 THEN '170<'
            WHEN team_imc.height_in_cm_mean BETWEEN 171 AND 175 THEN '171 - 175'
            WHEN team_imc.height_in_cm_mean BETWEEN 176 AND 180 THEN '176 - 180'
            WHEN team_imc.height_in_cm_mean BETWEEN 181 AND 185 THEN '181 - 185'
            ELSE '186>' END AS height_in_cm_mean_group,
        CASE
            WHEN team_imc.weight_in_kg_mean <= 70 THEN '70<'
            WHEN team_imc.weight_in_kg_mean BETWEEN 71 AND 75 THEN '71 - 75'
            WHEN team_imc.weight_in_kg_mean BETWEEN 76 AND 80 THEN '76 - 80'
            WHEN team_imc.weight_in_kg_mean BETWEEN 81 AND 85 THEN '81 - 85'
            ELSE '86>' END AS weight_in_kg_mean_group,
        COUNT(DISTINCT team_imc.team_api_id) AS teams,
        COUNT(DISTINCT dedup_matches.match_api_id) AS matches,
        SUM(dedup_matches.win) AS wins,
        ROUND((SUM(dedup_matches.win) * 1.0 / COUNT(DISTINCT dedup_matches.match_api_id) * 1.0) * 100) AS win_ratio
        
    FROM
        dedup_matches
    INNER JOIN
        team_imc
    ON dedup_matches.team_api_id = team_imc.team_api_id
    GROUP BY
        height_in_cm_mean_group,
        weight_in_kg_mean_group
    ORDER BY
        win_ratio DESC
''',conn)
df.head(30)

,height_in_cm_mean_group,weight_in_kg_mean_group,teams,matches,wins,win_ratio
0,181 - 185,76 - 80,167,19856,11552,58.0
1,176 - 180,71 - 75,10,1431,590,41.0
2,176 - 180,76 - 80,3,538,220,41.0
3,181 - 185,86>,48,6871,2739,40.0
4,186>,81 - 85,3,604,233,39.0
5,186>,86>,26,4121,1578,38.0
6,186>,71 - 75,15,2352,796,34.0
7,181 - 185,71 - 75,11,1581,518,33.0
8,186>,76 - 80,10,1537,456,30.0
9,181 - 185,81 - 85,4,462,124,27.0


A maior parte dos times possuem uma altura média entre 181 e 185 cm e pesam entre 76 e 80 kg e quando avaliamos o desempenho baseado em número de vítorias estes times vencem cerca de 17 p.p à mais que o segundo grupo melhor colocado.

In [63]:
conn.close()